In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/GitHub/Biological-Foundation-Model/Notebooks')

!pip install -r ../requirements.txt

  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


In [2]:
!cd "/content/drive/My Drive/Colab Notebooks/huggingface_models"
!pip freeze > "/content/drive/My Drive/Colab Notebooks/huggingface_models/requirements.txt"
%cd "/content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer"
!pip install .

/content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer
Processing /content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.2 MB/s eta 0:00:00
  Created wheel for geneformer: filename=geneformer-0.1.0-py3-none-any.whl size=2410396 sha256=a35b68b9121a8792823e153

In [ ]:
import sys
sys.path.append("structural_probe_utils")
import numpy as np
import pandas as pd

import torch
import anndata as ad

from structural_probe_utils.structural_probe_utils import build_G_from_edges, train_A

In [3]:
# prepare gene embedding

from geneformer import TOKEN_DICTIONARY_FILE
from geneformer import perturber_utils as pu
from geneformer import TranscriptomeTokenizer
model = pu.load_model("Pretrained", num_classes = 0, model_directory = "/content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer/gf-12L-30M-i2048", mode="eval")

In [4]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(25426, 512, padding_idx=0)
      (position_embeddings): Embedding(2048, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.02, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.02, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwis

In [ ]:
# prepare true gene network
STRING_homosapien = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Structural_Probe_Gene/data/9606.protein.links.v12.0.txt", sep=" ")
STRING_homosapien["protein1"] = STRING_homosapien["protein1"].str.replace("9606.", "", regex=False)
STRING_homosapien["protein2"] = STRING_homosapien["protein2"].str.replace("9606.", "", regex=False)

STRING_homosapien_alias = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Structural_Probe_Gene/data/9606.protein.aliases.v12.0.txt", sep="\t", header=None)
STRING_homosapien_alias = STRING_homosapien_alias[STRING_homosapien_alias[2].str.startswith("Ensembl_HGNC_symbol", na=False)][[0,1]].drop_duplicates()
STRING_homosapien_alias[0] = STRING_homosapien_alias[0].str.replace("9606.", "", regex=False)
ENSP_to_GeneSymbol = dict(zip(STRING_homosapien_alias[0], STRING_homosapien_alias[1]))

# Map both columns using the dictionary
STRING_homosapien["gene1"] = STRING_homosapien["protein1"].map(ENSP_to_GeneSymbol)
STRING_homosapien["gene2"] = STRING_homosapien["protein2"].map(ENSP_to_GeneSymbol)
STRING_gene_interaction = STRING_homosapien.dropna(subset=["gene1", "gene2", "combined_score"])
STRING_gene_interaction_high_conf = STRING_gene_interaction[STRING_gene_interaction["combined_score"]>700]

In [ ]:
G_full = build_G_from_edges(STRING_gene_interaction_high_conf, gene_names, make_dense = True)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score

n_splits = 10
n = len(gene_names)
random_state = 42
device = "cpu"


G_full = build_G_from_edges(STRING_gene_interaction_high_conf, gene_names, make_dense = True)

genes = np.array(gene_names)
X_full = np.asarray(gene_pos_emb)  # (n,d)
n = len(genes)

kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

acc_tr_0 = []
acc_tr_1 = []
acc_va_0 = []
acc_va_1 = []
f1_tr = []
f1_va = []
for fold_id, (train_idx, val_idx) in enumerate(kf.split(np.arange(n)), start=1):
    # Induced subgraphs and embeddings
    X_tr = X_full[train_idx]
    X_va = X_full[val_idx]
    G_tr = G_full[np.ix_(train_idx, train_idx)]
    G_va = G_full[np.ix_(val_idx,  val_idx)]

    # Train
    A, w, tr_loss, va_loss = train_A(
        X_tr, G_tr, X_va, G_va, steps = 200
    )

    # X_tr = torch.as_tensor(X_tr, dtype=torch.float32, device=device)
    # np.fill_diagonal(G_tr, 0)
    # G_tr = torch.from_numpy((G_tr > 0).astype(np.bool_)).to(device)
    # nt = X_tr.shape[0]
    # I, J = torch.triu_indices(nt, nt, offset=1, device=device)
    # y = G_tr[I, J].float()

    X_va = torch.as_tensor(X_va, dtype=torch.float32, device=device)
    np.fill_diagonal(G_va, 0)
    G_va = torch.from_numpy((G_va > 0).astype(np.bool_)).to(device)
    nv = X_va.shape[0]
    Iv, Jv = torch.triu_indices(nv, nv, offset=1, device=device)
    yv = G_va[Iv, Jv].float()

    with torch.no_grad():
      # delta = X_tr[I] - X_tr[J]
      # z = delta @ A
      # sqdist = (z * z).sum(dim=1)
      # logits = w - sqdist
      # prob = torch.sigmoid(logits).cpu().numpy()
      # pred = (prob > 0.5).astype(int)

      vdelta = X_va[Iv] - X_va[Jv]
      vz = vdelta @ A
      vsqdist = (vz * vz).sum(dim=1)
      vlogits = w - vsqdist
      vprob = torch.sigmoid(vlogits).cpu().numpy()
      vpred = (vprob > 0.5).astype(int)

      # acc_tr_0.append(accuracy_score(y[y == 0], pred[y == 0]))
      # acc_tr_1.append(accuracy_score(y[y == 1], pred[y == 1]))
      acc_va_0.append(accuracy_score(yv[yv == 0], vpred[yv == 0]))
      acc_va_1.append(accuracy_score(yv[yv == 1], vpred[yv == 1]))
      # f1_tr.append(f1_score(y, pred))
      f1_va.append(f1_score(yv, vpred))


In [ ]:
print(acc_va_0)
print(acc_va_1)
print(f1_va)